In [145]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib
import seaborn as sns
import csv
import re 
from matplotlib import pyplot as plt
from sklearn import ensemble, cross_validation, learning_curve, metrics , tree, naive_bayes, utils
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.datasets import load_digits, load_breast_cancer, load_iris
from sklearn.cross_validation import cross_val_score, KFold, LeaveOneOut, ShuffleSplit, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier, BaggingRegressor
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans, AgglomerativeClustering, FeatureAgglomeration
from sklearn.preprocessing import StandardScaler, scale, OneHotEncoder
matplotlib.style.use('ggplot')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [146]:
structure = pd.read_csv('course-217-structure.csv')
events = pd.read_csv('course-217-events.csv')

In [148]:
for row in structure:
    if len( structure[row].unique() ) < 20:
        print(row,  structure[row].unique() )

course_id [217]
module_id [ 614  618  619  620  621  622  623  624  635 1260]
module_position [ 2  4  3  6  5  7  8  9  1 10]
lesson_position [2 3 4 1 5 6 7 8 9]
step_position [ 1  6  7  8  9  4  2  3  5 10 11 12 13]
step_type ['text' 'code' 'free-answer' 'video' 'choice' 'sorting' 'matching']
step_cost [0 1]


In [149]:
events.head(10)

,user_id,action,step_id,time
0,1,viewed,47435,1465181036
1,1,viewed,47438,1465181034
2,1,viewed,47435,1465181021
3,1,viewed,47436,1465181015
4,1,viewed,41605,1465180037
5,1,viewed,47436,1465180033
6,1,viewed,47436,1465179924
7,2,viewed,38906,1465176914
8,2,viewed,38909,1465176484
9,3,viewed,44115,1465176471


In [152]:
events_sorted = events.sort_values('time', axis=0)

In [158]:
scorring_events = events[events['action'] == 'passed']
print(scorring_events.shape)

(274752, 4)


In [159]:
scorring_events_sorted = scorring_events.sort_values('time', axis=0)
scorring_events_sorted.to_csv('scorring_events_sorted.csv')

In [157]:
scorring_events_sorted.head(50)

,user_id,action,step_id,time
1431920,1229,passed,38518,1440065573
1431917,1229,passed,38521,1440065578
1431908,1229,passed,38527,1440065595
1431905,1229,passed,38522,1440065597
1431902,1229,passed,38523,1440065607
1431899,1229,passed,38524,1440065617
1431896,1229,passed,38525,1440065620
1431893,1229,passed,38528,1440065661
1431890,1229,passed,38526,1440065663
1431887,7662,passed,40459,1440584595


In [161]:
time_to_begin_course = {}
for user_id, _, __, time in events_sorted.values:
    if user_id in time_to_begin_course:
        pass
    else:
        time_to_begin_course[user_id] = time

In [167]:
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_id,step_position,step_type,step_cost
0,217,614,2,13228,2,38842,1,text,0
1,217,614,2,13228,2,39715,6,code,1
2,217,614,2,13228,2,39716,7,code,1
3,217,614,2,13228,2,39717,8,code,1
4,217,614,2,13228,2,39721,9,free-answer,0


In [169]:
def get_cost(step_id):
    for row in structure.values:
        if row[5] == step_id:
            return row[8]

In [178]:
users_achievment = {}
top_users = []
top_and_time = []
for user_id, _, step_id, time in scorring_events_sorted.values:
    if user_id in users_achievment:
        users_achievment[user_id] += get_cost(step_id)
        if users_achievment[user_id] >= 24:
            if user_id not in top_users:
                top_users.append(user_id)
                top_and_time.append([user_id, time - time_to_begin_course[user_id]])
    else:
        users_achievment[user_id] = get_cost(step_id)
print(list(map(lambda x: x[0], sorted(top_and_time, key=lambda x: x[1])[0:10])))

[4313, 1264, 821, 1146, 1124, 2622, 713, 972, 4280, 1291]


In [24]:
def write_answer(y_pred, col_id):
    answer_df = pd.concat([col_id, y_pred], axis=1)
    answer_df.to_csv('my_submission')